In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pickle import dump
def stock(name):
  df=web.DataReader(name,data_source='yahoo',start='2012-01-01', end='2021-06-07')
  data=df.filter(['Close'])
  dataset=data.values
  training_data_len=math.ceil(len(dataset)*0.8)
  scaler=MinMaxScaler(feature_range=(0,1))
  scaled_data= scaler.fit_transform(dataset)
  train_data=scaled_data[0:training_data_len,:]
  x_train=[]
  y_train=[]
  for i in range(60, len(train_data)):
   x_train.append(train_data[i-60:i,0])
   y_train.append(train_data[i,0])
  x_train,y_train=np.array(x_train),np.array(y_train)
  x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
  model=Sequential()
  model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
  model.add(LSTM(50,return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))
  model.compile(optimizer='adam',loss='mean_squared_error')
  model.fit(x_train,y_train,batch_size=1,epochs=3)
  # test_data=scaled_data[training_data_len-60:,:]
  # x_test=[]
  # y_test=dataset[training_data_len:,:]
  # for i in range(60,len(test_data)):
  #   x_test.append(test_data[i-60:i,0])
  # x_test=np.array(x_test)
  # x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
  # predictions=model.predict(x_test)
  # predictions=scaler.inverse_transform(predictions)
  # rmse=np.sqrt(np.mean(predictions-y_test)**2)
  # train=data[0:training_data_len]
  # valid=data[training_data_len:]
  # valid['Predictions']=predictions
  # valid['Date']=np.array(valid.index)
  # plt.figure(figsize=(16,8))
  # plt.title('Model')
  # plt.xlabel('Data',fontsize=18)
  # plt.ylabel('Close Price USD ($)',fontsize=18)
  # plt.plot(train['Close'])
  # plt.plot(valid[['Close','Predictions']])
  # plt.legend(['Train','Validation', 'Prediction'], loc ='lower right')
  model.save('TATAMOTORS_model.h5')
  dump(scaler,open("TATAMOTORS_scaler.pkl","wb"))
stock('TATAMOTORS.NS')


  


In [ ]:
import datetime
today = datetime.date.today()
span2 = datetime.timedelta(days=2)
span1 = datetime.timedelta(days=1)
Prev_2_date=today-span2
Prev_1_date=today-span1

def test_data(name):
  df=web.DataReader(name,data_source='yahoo',start='2012-01-01', end=str(Prev_2_date))
  data=df.filter(['Close'])
  dataset=data.values
  training_data_len=math.ceil(len(dataset)*0.8)
  scaled_data= scaler.transform(dataset)
  test_data=scaled_data[training_data_len-60:,:]
  x_test=[]
  y_test=dataset[training_data_len:,:]
  for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])
  x_test=np.array(x_test)
  x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
  predictions=rmodel.predict(x_test)
  predictions=scaler.inverse_transform(predictions)
  rmse=np.sqrt(np.mean(predictions-y_test)**2)
  valid=data[training_data_len:]
  valid['Predictions']=predictions
  print((valid[-15:]))
test_data('TCS.NS')



In [ ]:
 import datetime
 today = datetime.date.today()
 def predict(name): 
  quote=web.DataReader(name,data_source='yahoo',start='2012-01-01', end=(Prev_1_date))
  new_df=quote.filter(['Close'])
  last_60_days=new_df.iloc[-60:].values
  last_60_days_scaled=scaler.transform(last_60_days)
  X_test=[]
  X_test.append(last_60_days_scaled)
  X_test=np.array(X_test)
  X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
  pred_price=rmodel.predict(X_test)
  pred_price=scaler.inverse_transform(pred_price)
  print(np.array(pred_price))
predict('TCS.NS')

[[3094.646]]


In [ ]:
pip install flask-ngrok


In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import math
import pandas_datareader as web
import numpy as np
from requests.sessions import session
from pickle import load
from tensorflow.keras.models import load_model
import datetime
today = datetime.date.today()
span30 = datetime.timedelta(days=30)
span2 = datetime.timedelta(days=2)
span1 = datetime.timedelta(days=1)
Next_30_date=today+span30
Prev_2_date=today-span2
Prev_1_date=today-span1
next_date=today+span1

app=Flask(__name__)
run_with_ngrok(app)



def predict(name,model,scaler): 
  quote=web.DataReader(name,data_source='yahoo',start='2012-01-01', end=str(Prev_1_date))
  new_df=quote.filter(['Close'])
  last_60_days=new_df.iloc[-60:].values
  last_60_days_scaled=scaler.transform(last_60_days)
  X_test=[]
  X_test.append(last_60_days_scaled)
  X_test=np.array(X_test)
  X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
  pred_price=model.predict(X_test)
  pred_price=scaler.inverse_transform(pred_price)
  return (pred_price)
def last15dates():
  l=[]
  for i in range(15):
    span= datetime.timedelta(days=i)
    Prevdate=today-span
    l.append(Prevdate)
  return l

@app.route('/', methods=["GET"])
def main():
    result={
        "Bitcoin":"/CRYPTO/BITCOIN",
        "NIFT 50 Stocks":"/STOCKS/NIFTY50",
        "NIFT BANK Stocks":"/STOCKS/NIFTYBANK",
        "NIFT IT Stocks":"/STOCKS/NIFTYIT",
    }
    return jsonify(result)

@app.route('/CRYPTO/BITCOIN')
def bitcoin():
    url='https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters={
        'start':'1',
        'limit':'1',
        'convert':'USD',

    }
    headers={
        'Acecpts': 'application/json',
        'X-CMC_PRO_API_KEY':'944f669d-87f0-4a45-a700-4e04d3b820b6',
    }
    session= Session()
    session.headers.update(headers)
    try:
        response=session.get(url,params=parameters)
        data=json.loads(response.text)
        price=data['data'][0]['quote']['USD']['price']
        price_array=np.array([[price]])
        #print(price_array)
    except(ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    loaded_model=load(open("bitcoin_predictor.pkl","rb"))
    #print(loaded_model.predict(price_array))
    result={
        "Type":"Bit-Coin",
        str(today):int(price_array),
        str(Next_30_date):int(loaded_model.predict(price_array))
    }
    return jsonify(result)

@app.route('/STOCKS/NIFTY50')
def NIFTY50():
    model=load_model("NIFTY50_model.h5")
    scaler=load(open("NIFTY50_scaler.pkl","rb"))
    next=predict('^NSEI',model,scaler)
    dte=last15dates()
    
    result={
        "Type":"Stock-NIFTY-50",
        str(next_date):int(next)
    }
    return jsonify(result)
@app.route('/STOCKS/NIFTYBANK')
def NIFTYBANK():
    model=load_model("NIFTYBANK_model.h5")
    scaler=load(open("NIFTYBANK_scaler.pkl","rb"))
    next=predict('^NSEBANK',model,scaler)
    dte=last15dates()
    
    result={
        "Type":"Stock-NIFTY-BANK",
        str(next_date):int(next)
    }
    return jsonify(result)
@app.route('/STOCKS/NIFTYIT')
def NIFTYIT():
    model=load_model("NIFTYIT_model.h5")
    scaler=load(open("NIFTYIT_scaler.pkl","rb"))
    next=predict('^CNXIT',model,scaler)
    dte=last15dates()
    
    result={
        "Type":"Stock-NIFTY-IT",
        str(next_date):int(next)
    }
    return jsonify(result)



     

if __name__=="__main__":
    app.run()

ModuleNotFoundError: ignored

In [ ]:
# TECHMAHINDRA_model=load_model("TECHMAHINDRA_model.h5")
# TECHMAHINDRA_scaler=load(open("TECHMAHINDRA_scaler.pkl","rb"))
# BHARTIAIRTEL_model=load_model("BHARTIAIRTEL_model.h5")
# BHARTIAIRTEL_scaler=load(open("BHARTIAIRTEL_scaler.pkl","rb"))
# TATAMOTORS_model=load_model("TATAMOTORS_model.h5")
# TATAMOTORS_scaler=load(open("TATAMOTORS_scaler.pkl","rb"))
# TCS_model=load_model("TCS_model.h5")
# TCS_scaler=load(open("TCS_scaler.pkl","rb"))

# @app.route('/STOCKS/TCS')
# def TCS():
#     model=TCS_model
    
#     test=test_data('TCS.NS')
#     next=predict('TCS.NS')
#     dte=last15dates()
    
#     result={
#         "Type":"Stock-TCS",
#         str(next_date):int(next)
#     }
#     return jsonify(result)
# @app.route('/STOCKS/TECHMAHINDRA')
# def TECHMAHINDRA():
#     model=TECHMAHINDRA_model
    
#     test=test_data('TECHM.NS')
#     next=predict('TECHM.NS')
#     dte=last15dates()
    
#     result={
#         "Type":"Stock-TECH-MAHINDRA-LIMITED",
#         str(next_date):int(next)
#     }
#     return jsonify(result)
# @app.route('/STOCKS/BHARTIAIRTEL')
# def BHARTIAIRTEL():
#     model=BHARTIAIRTEL_model
    
#     test=test_data('BHARTIARTL.NS')
#     next=predict('BHARTIARTL.NS')
#     dte=last15dates()
    
#     result={
#         "Type":"Stock-BHARTI-AIRTEL",
#         str(next_date):int(next)
#     }
#     return jsonify(result)
# @app.route('/STOCKS/TATAMOTORS')
# def TATAMOTORS():
#     model=TATAMOTORS_model
    
#     test=test_data('TATAMOTORS.NS')
#     next=predict('TATAMOTORS.NS')
#     dte=last15dates()
    
#     result={
#         "Type":"Stock-TATA-MOTORS",
#         str(next_date):int(next)
#     }
#     return jsonify(result)
